In [1]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving data55.xlsx to data55.xlsx


In [118]:
data = pd.read_excel('data55.xlsx', header = None)             # reading the data
data = data.sample(frac = 1, random_state = 42).reset_index(drop = True)    # shuffling the data
data.head(10)

,0,1,2,3,4
0,6.0,2.7,5.1,1.6,1
1,5.5,2.3,4.0,1.3,1
2,5.9,3.2,4.8,1.8,1
3,4.8,3.0,1.4,0.3,0
4,5.1,3.8,1.9,0.4,0
5,5.1,3.4,1.5,0.2,0
6,4.6,3.6,1.0,0.2,0
7,5.5,2.4,3.8,1.1,1
8,5.4,3.7,1.5,0.2,0
9,5.1,3.5,1.4,0.2,0


In [119]:
## separating input(X) and output(Y)
X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]
print("First 5 rows of X")
print(X.head())
print()
print("First 5 rows of Y")
print(Y.head())

First 5 rows of X
     0    1    2    3
0  6.0  2.7  5.1  1.6
1  5.5  2.3  4.0  1.3
2  5.9  3.2  4.8  1.8
3  4.8  3.0  1.4  0.3
4  5.1  3.8  1.9  0.4

First 5 rows of Y
   4
0  1
1  1
2  1
3  0
4  0


In [120]:
### splitting data in train, validation and test set
def split(X, Y, val_split = 0.1, test_split = 0.2):
  '''Arguments:
      X, Y: Input and Output data
      val_split: % of data for validation set
      test_split: % of data for test set
      Returns: train, validation and test set data in the following order - X_train, Y_train, X_val, Y_val, X_test, Y_test
      '''
  train_size = int(len(X) * (1 - (val_split + test_split)))
  val_size = int(len(X) * val_split)
  #test_size = int(len(X) * test_split)
  X_tr = X.iloc[:train_size, :]
  Y_tr = Y.iloc[:train_size, :]
  X_val = X.iloc[train_size:train_size + val_size, :].reset_index(drop = True)
  Y_val = Y.iloc[train_size:train_size + val_size, :].reset_index(drop = True)
  X_ts = X.iloc[train_size + val_size:, :].reset_index(drop = True)
  Y_ts = Y.iloc[train_size + val_size:, :].reset_index(drop = True)

  return X_tr, Y_tr, X_val, Y_val, X_ts, Y_ts

In [121]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = split(X, Y, val_split = 0.1, test_split = 0.2)
print("X_train shape is: ", X_train.shape)
print("Y_train shape is: ", Y_train.shape)
print("X_val shape is: ", X_val.shape)
print("Y_val shape is: ", Y_val.shape)
print("X_test shape is: ", X_test.shape)
print("Y_test shape is: ", Y_test.shape)

X_train shape is:  (70, 4)
Y_train shape is:  (70, 1)
X_val shape is:  (10, 4)
Y_val shape is:  (10, 1)
X_test shape is:  (20, 4)
Y_test shape is:  (20, 1)


In [122]:
def normalize(data, train_data):
  '''
  data: data that is to be normalized
  train_data: normalize the data using training_data features
  Func returns normalized data
  '''
  norm_data = (data - train_data.mean(axis = 0))/train_data.std(axis = 0)
  return norm_data

In [123]:
#### normalizing data and converting to array
X_train_norm = np.array(normalize(X_train, X_test))
X_val_norm = np.array(normalize(X_val, X_train))
X_test_norm = np.array(normalize(X_test, X_train))

In [124]:
Y_train = np.array(Y_train).reshape(Y_train.shape[0],)
Y_val = np.array(Y_val).reshape(Y_val.shape[0],)
Y_test = np.array(Y_test).reshape(Y_test.shape[0],)

In [125]:
def kernel_rbf(x,z,s=1):
  trnorms1 = np.mat([np.dot(v,v.T) for v in x]).T
  trnorms2 = np.mat([np.dot(v,v.T) for v in z]).T
  k1 = trnorms1 * np.mat(np.ones((z.shape[0], 1), dtype=np.float64)).T
  k2 = np.mat(np.ones((x.shape[0], 1), dtype=np.float64)) * trnorms2.T
  k = k1 + k2
  k -= 2 * np.dot(x,z.T)
  k *= - 1./(np.power(s, 2))
  return np.exp(k)

In [132]:
def kernel_perceptron(a,train_X,y,iter):
  StopCondition=1
  Y=np.empty(len(Y_train),int)
  Y_prev=Y
  K=kernel_rbf(X_train_norm,X_train_norm)
  #print(K)
  for it in range(iter):
    #print(it)
    for j in range(X_train.shape[0]):
      temp=a*Y_train
      sum=np.dot(K[j,:],temp)
      Y[j]=np.where(sum<=0,-1,1)
      #print(Y[j])
      a[j]=a[j]+np.where(Y[j]!=Y_train[j],1,0)
    
    isEqual = np.sum(Y == Y_prev) / len(Y);
    #print(isEqual)
    if (isEqual >= StopCondition):
      break;
    Y_prev=Y
  return a,K

In [127]:
### Training accuracy
np.random.seed(0)
w=np.random.randn(train_X.shape[1])  
b=np.random.random(1)

a=np.zeros(X_train_norm.shape[0])
y=np.empty(len(Y_train),int)

for i in range(len(Y_train)):
  y[i]=np.where(Y_train[i]==0,-1,1)

w,K=kernel_perceptron(a,X_train_norm,y,1)
train_pred=np.empty(len(Y_train),int)
for j in range(Y_train.shape[0]):
      temp=a*y
      sum=np.sum(temp*K[j,:].T)
      train_pred[j]=np.where(sum<=0,-1,1)

trainacc=np.sum(train_pred==y)/len(y)
print(f"Training accurcay is {trainacc*100} %")

Training accurcay is 98.57142857142858 %


In [134]:
### performance measure
def performance_measure(Y_act, Y_pred):
  conf_mat = pd.crosstab(Y_act, Y_pred)
  acc = (conf_mat[-1][-1] + conf_mat[1][1])/len(Y_act)
  sen = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[-1][1])
  speci = conf_mat[-1][-1]/(conf_mat[-1][-1]+conf_mat[1][-1])
  print(conf_mat)

  print(f"accuracy is {acc*100} %")
  print(f"sensitivity is {sen*100} %")
  print(f"specificity is {speci*100} %")

In [133]:
#### predicting labels for validation set
tval=np.empty(len(Y_val),int)
for i in range(len(Y_val)):
  tval[i]=np.where(Y_val[i]==0,-1,1)

Kval=kernel_rbf(X_val_norm,X_train_norm)
#print(K1.shape)
val_pred=np.empty(len(X_val_norm),int)
for j in range(X_val_norm.shape[0]):
  temp=w*y
  #print(temp.shape)
  sum=np.sum(temp*Kval[j,:].T)
  val_pred[j]=np.where(sum<=0,-1,1)

In [135]:
### measuring performance of validation set
performance_measure(tval, val_pred)

col_0  -1   1
row_0        
-1      3   0
 1      0   7
accuracy is 100.0 %
sensitivity is 100.0 %
specificity is 100.0 %


In [129]:
### predicting labels for test set
ty=np.empty(len(Y_test),int)
for i in range(len(Y_test)):
  ty[i]=np.where(Y_test[i]==0,-1,1)

K1=kernel_rbf(X_test_norm,X_train_norm)
#print(K1.shape)
test_pred=np.empty(len(X_test_norm),int)
for j in range(X_test_norm.shape[0]):
  temp=w*y
  #print(temp.shape)
  sum=np.sum(temp*K1[j,:].T)
  test_pred[j]=np.where(sum<=0,-1,1)

In [131]:
### measuring performance of test set
performance_measure(ty, test_pred)

col_0  -1   1
row_0        
-1      8   0
 1      0  12
accuracy is 100.0 %
sensitivity is 100.0 %
specificity is 100.0 %
